In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append('%s/../' % os.getcwd())

import datetime
import pandas as pd
import numpy as np
import math

from pyxlsb import open_workbook as open_xlsb

In [2]:
class AnaliseQuebrasBMFBovespa():
    path = r'\\sp-desen-0002\Shared\Guilherme'
    archive = r'\Allocation-'
    
    def __init__(self, data_referencia):
        self.data_referencia = data_referencia
        self.fullpath = AnaliseQuebrasBMFBovespa.path+AnaliseQuebrasBMFBovespa.archive+data_referencia+'.xlsb'
        self.df = self.open_xlsb_file()
        
    def open_xlsb_file(self):
        df = []
        with open_xlsb(self.fullpath) as wb:
            with wb.get_sheet(1) as sheet:
                for row in sheet.rows():
                    df.append([item.v for item in row])

        df = pd.DataFrame(df[1:], columns=df[0])
        return df
    
    
    @staticmethod
    def agrupa_grouped_dataframes(df, column_list, merging_columns, final_name_dictionary):
        output = df.merge(df.groupby(column_list).sum().reset_index()[sum([column_list,merging_columns],[])]\
                          .rename(columns=final_name_dictionary), on=column_list, how='left')
        shape_diff = df.shape[0]-output.shape[0]

        if shape_diff!=0:
            print('Houve breaks no merge. Dfs com shapes diferentes (Diff: %s linhas)' %(shape_diff))
        return output
    
    @staticmethod
    def remove_duplicates(values):
        output = []
        seen = set()
        for value in values:
            # If value has not been encountered yet,
            # ... add it to both list and set.
            if value not in seen:
                output.append(value)
                seen.add(value)
        return output

In [3]:
def agrupa_grouped_dataframes(df, column_list, merging_columns, final_name_dictionary):
        output = df.merge(df.groupby(column_list).sum().reset_index()[sum([column_list,merging_columns],[])]\
                          .rename(columns=final_name_dictionary), on=column_list, how='left')
        shape_diff = df.shape[0]-output.shape[0]

        if shape_diff!=0:
            print('Houve breaks no merge. Dfs com shapes diferentes (Diff: %s linhas)' %(shape_diff))
        return output
    
def remove_duplicates(values):
    output = []
    seen = set()
    for value in values:
        # If value has not been encountered yet,
        # ... add it to both list and set.
        if value not in seen:
            output.append(value)
            seen.add(value)
    return output

In [4]:
g = AnaliseQuebrasBMFBovespa('20180906')

In [5]:
df = g.df

## Estou tentando tirar as realocações abaixo

In [6]:
chave_grupo = ['Date','Symbol','ClearingTradeId','Side']

In [7]:
df = df[df['ClearingFirm']==85].sort_values(by=['Date','Symbol','ClearingTradeId','Side',
                                                'TransactTime']).reset_index(drop=True)

In [8]:
df.loc[:,'TipoMasterAccount'] = np.where(df['AccountCode']==df['MasterAccount'], 1,0)

In [9]:
# Coluna é = 1 quando a linha é o primeiro trade com aquelas características
df.loc[:,'PrimeiroLancamentoTrade'] = np.where((df['ClearingTradeId']!=df['ClearingTradeId'].shift(1))|\
                                              (df['Side']!=df['Side'].shift(1)),1,0)
# Tem mais de um account
df.loc[:,'MudaDeAccDentrodoTrade'] = np.where((df['Date']==df['Date'].shift(1))&
                                              (df['Symbol']==df['Symbol'].shift(1))&
                                              (df['ClearingTradeId']==df['ClearingTradeId'].shift(1))&
                                              (df['Side']==df['Side'].shift(1))&
                                              (df['AccountCode']!=df['AccountCode'].shift(1)),1,0)
df.loc[:,'ContaErro'] = np.where((df['MasterAccount']==99999.0)|
                                (df['MasterAccount']==15000.0), 1,0)

In [10]:
df = agrupa_grouped_dataframes(df,['Date','Symbol','ClearingTradeId','Side'],['TipoMasterAccount', 'ContaErro',
                                                                              'MudaDeAccDentrodoTrade'],
                               {'TipoMasterAccount':'TotalMasterAccount',
                                'MudaDeAccDentrodoTrade':'Total_TrocaAcc',
                                'ContaErro':'PassouPelaContaErro'})

In [13]:
df = df[df['TotalMasterAccount']>1]

In [17]:
df[sum([chave_grupo,['MasterAccount','AccountCode','OriginalQty','Qty']],[])]

,Date,Symbol,ClearingTradeId,Side,MasterAccount,AccountCode,OriginalQty,Qty
530,43349.0,ARZZ3,70.0,2.0,100183.0,100183.0,0.0,0.0
531,43349.0,ARZZ3,70.0,2.0,999985.0,999985.0,0.0,0.0
532,43349.0,ARZZ3,70.0,2.0,100183.0,133753.0,5300.0,5300.0
1472,43349.0,BIDI4,36590.0,2.0,98595.0,98595.0,100000.0,0.0
1473,43349.0,BIDI4,36590.0,2.0,999985.0,999985.0,0.0,0.0
1474,43349.0,BIDI4,36590.0,2.0,98595.0,200557.0,4431.0,4431.0
1475,43349.0,BIDI4,36590.0,2.0,98595.0,478264.0,10778.0,10778.0
1493,43349.0,BIDI4,39290.0,2.0,98595.0,98595.0,95400.0,0.0
1494,43349.0,BIDI4,39290.0,2.0,999985.0,999985.0,0.0,0.0
1495,43349.0,BIDI4,39290.0,2.0,98595.0,207494.0,56031.0,56031.0


In [ ]:
df = df[df['ClearingFirm']==85].sort_values(by=['Date','Symbol','ClearingTradeId','Side','TransactTime']).reset_index(drop=True)

df_backup = df.copy()
#df = df_backup.copy()



# Tipo acc Master = 1 ou filhote = 0
df.loc[:,'TipoMasterAccount'] = np.where(df['AccountCode']==df['MasterAccount'], 1,0)
# Coluna de Count
df.loc[:,'Count'] = 1

# Coluna é = 1 quando a linha é o primeiro trade com aquelas características
df.loc[:,'PrimeiroLancamentoTrade'] = np.where((df['ClearingTradeId']!=df['ClearingTradeId'].shift(1))|\
                                              (df['Side']!=df['Side'].shift(1)),1,0)
# Tem mais de um account
df.loc[:,'MudaDeAccDentrodoTrade'] = np.where((df['Date']==df['Date'].shift(1))&
                                              (df['Symbol']==df['Symbol'].shift(1))&
                                              (df['ClearingTradeId']==df['ClearingTradeId'].shift(1))&
                                              (df['Side']==df['Side'].shift(1))&
                                              (df['AccountCode']!=df['AccountCode'].shift(1)),1,0)

df.loc[:,'ContaErro'] = np.where((df['MasterAccount']==99999.0)|
                                (df['MasterAccount']==15000.0), 1,0)



df = agrupa_grouped_dataframes(df,['Date','Symbol','ClearingTradeId','Side','TipoMasterAccount'],['OriginalQty','Qty',
                                                                                                  'ContaErro',
                                                                                                  'MudaDeAccDentrodoTrade'],
                               {'OriginalQty':'OriginaQty_TipoAcc', 'Qty':'Qty_TipoAcc','ContaErro':'PassouPelaContaErro',
                               'MudaDeAccDentrodoTrade':'Total_TrocaAcc'})
#df = agrupa_grouped_dataframes(df,['Date','Symbol','ClearingTradeId','Side','AccountCode'],['OriginalQty'],
#                               {'OriginalQty':'OriginaQty_PorAcc'})
#df = agrupa_grouped_dataframes(df,['Date','Symbol','ClearingTradeId','Side','MasterAccount'],['OriginalQty'],
#                               {'OriginalQty':'OriginaQty_PorMasterAcc'})
df = agrupa_grouped_dataframes(df, chave_grupo, ['Count'],{'Count':'TotalLinha_Trade'})

In [ ]:
# Faz a soma cumulativa das accounts por trade
df['MasterAccountAsTypeSTR'] = df['MasterAccount'].astype(str)
df.loc[:,'MasterAccountCumSum'] = df.groupby(chave_grupo)['MasterAccountAsTypeSTR'].apply(lambda x: (x+';').cumsum().str.strip())
df.loc[:,'MasterAccountDropDuplicates'] = df['MasterAccountCumSum'].apply(lambda x: ';'.join(remove_duplicates(x.split(';'))))
df.loc[:,'Trade_Realocado'] = np.where(df['MasterAccountCumSum']=='100183.0;999985.0;100183.0;',1,0)
df = agrupa_grouped_dataframes(df, chave_grupo, ['Trade_Realocado'],{'Trade_Realocado':'Trade_Realocado_Grupo'})



In [ ]:
df = df[(df['Trade_Realocado_Grupo']!=1)]

In [ ]:
df.loc[:,'Break_TradeNull'] = np.where((df['TotalLinha_Trade']==2)&
                                      (df['PassouPelaContaErro']==1)&
                                      (df['OriginalQty']==0)&(df['Qty']==0)&
                                       ((df['TransactTime']-df['TransactTime'].shift(1))*1000<1),1,0)
#df.loc[:,'Break_MasterNAlocada'] = np.where((df['TotalLinha_Trade']==2)&
#                                      ((df['TransactTime']-df['TransactTime'].shift(1))*1000<1),1,0)
#df.loc[:,'Break_Realocacao'] = np.where((df['OriginaQty_TipoAcc']!=df['Qty_TipoAcc'])&
#                                        (df['TotalLinha_Trade']>2),1,0)
df.loc[:,'Break_Realocacao'] = np.where((df['MasterAccountDropDuplicates'].str.split(';').shape[0]>2)&
                                        (df['TotalLinha_Trade']>2),1,0)
#df.loc[:,'Realocacao'] = np.where(df['Trade_Realocado_Grupo']!=0,1,0)

In [ ]:
df = agrupa_grouped_dataframes(df, chave_grupo, ['Break_TradeNull','Break_Realocacao'],{
    'Break_TradeNull':'Trades_Break_TradeNull',
    'Break_Realocacao':'Trades_Break_Realocacao'
})
#'Break_MasterNAlocada':'Trades_Break_MasterNAlocada'

In [ ]:
df[(df['ClearingTradeId']==8920)&
         (df['Symbol']=='BPAC11')]#[sum([chave_grupo,['Qty','OriginalQty','MasterAccount','AccountCode',
                                  #                  'OriginaQty_TipoAcc','Qty_TipoAcc',
                                  #                  'OriginaQty_PorAcc','OriginaQty_PorMasterAcc']],[])]

In [ ]:
df = df[((df['Trades_Break_TradeNull']==1)|
        (df['Trades_Break_Realocacao']>=1))]

In [ ]:
df.shape

In [ ]:
df.to_excel('primeiro_resultado.xlsx')

In [ ]:
df_backup[(df_backup['ClearingTradeId']==50240)&
         (df_backup['Symbol']=='VVAR11')]

In [ ]:
df[(df['ClearingTradeId']==35530)&
         (df['Symbol']=='CCRO3')]

In [ ]:
df[(df['Symbol']=='GNDI3')]